# Yfinance_Usecase 
- developing python definition funneled into langchain tool thus to be selected  

In [27]:
from typing import Type
from langchain.llms import OpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field

from datetime import datetime, timedelta

import yfinance as yf

In [38]:
def get_current_stock_price(ticker):
    """Method to get current stock price"""

    ticker_data = yf.Ticker(ticker)
    recent = ticker_data.history(period="6m")
    return {"price": recent.iloc[0]["Close"], \
            #"currency": ticker_data.info["currency"]
            }

def get_stock_performance(ticker, past_date):
    """Method to get stock price change in percentage"""
    
    ticker_data = yf.Ticker(ticker)
    history = ticker_data.history(start=past_date)
    old_price = history.iloc[0]["Close"]
    current_price = history.iloc[-1]["Close"]
    return {"percent_change": ((current_price - old_price) / old_price) * 100}

class CurrentStockPriceInput(BaseModel):
    """Inputs for get_current_stock_price"""

    ticker: str = Field(description="Ticker symbol of the stock")


class CurrentStockPriceTool(BaseTool):
    name = "get_current_stock_price"
    description = """
        Useful when you want to get current stock price.
        You should enter the stock ticker symbol recognized by the yahoo finance
        """
    args_schema: Type[BaseModel] = CurrentStockPriceInput

    def _run(self, ticker: str):
        price_response = get_current_stock_price(ticker)
        return price_response

    def _arun(self, ticker: str):
        raise NotImplementedError("get_current_stock_price does not support async")



In [39]:
class StockPerformanceInput(BaseModel):
    """Inputs for get_stock_performance"""

    ticker: str = Field(description="Ticker symbol of the stock")
    past_date: str = Field(description="past_date asked to compare the price for the stock")


class StockPerformanceTool(BaseTool):
    name = "get_stock_performance"
    description = """
        Useful when you want to get the stock performance over the given period.
        You should enter the stock ticker symbol recognized by the yahoo finance
        """
    args_schema: Type[BaseModel] = StockPerformanceInput

    def _run(self, ticker: str , past_date: str):
        performance_response = get_stock_performance(ticker,past_date)
        return performance_response

    def _arun(self, ticker: str , past_date: str):
        raise NotImplementedError("get_stock_performance does not support async")

In [40]:
from langchain.agents import AgentType, initialize_agent, Tool
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.tools.tavily_search import TavilySearchResults
from langchain.utilities.tavily_search import TavilySearchAPIWrapper

In [41]:
from dotenv import load_dotenv
load_dotenv()

True

In [42]:
llm = ChatOpenAI(model="gpt-4", temperature=0, openai_api_key="")

search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search, max_results=10)
tools = [CurrentStockPriceTool(),StockPerformanceTool(),
         Tool(
        name="Search",
        func=tavily_tool.run,
        description="useful for when you need to get ticker and current event about the stock in question"
    )
]

agent = initialize_agent(tools=tools,
                          llm=llm,
                          agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
#(최근 거래가, 공매도일자, 수익률)
agent.run("get me 3 data \
          [1. the latest trading day stock price of Ecopro(KOSDAQ),\
           2. the date in which Korea totally banned short-selling,\
           3. its performance since the ban].\n"
          "Do not assume any random dates if the article searched doesn't say about it, you can tell me\
            that article's release date and ultimately the performance since then")



> Entering new AgentExecutor chain...
Thought: First, I need to find the ticker symbol for Ecopro on the KOSDAQ. I will use the Search tool for this. 
Action:
```
{
  "action": "Search",
  "action_input": "Ecopro KOSDAQ ticker symbol"
}
```
Observation: [{'url': 'https://finance.yahoo.com/quote/086520.KQ/', 'content': "Ecopro Co., Ltd. (086520.KQ) Subscribe to Yahoo Finance Plus to view Fair Value for 086520.KQ  Ecopro Materials eyes $498 million in potentially largest South Korean IPO since early 2022  Ecopro, Samsung Ink Massive $34 Billion Battery Deal Despite EV Slump  South Korea's Ecopro Materials, a battery materials producer, is gearing up for what could be the country's largestFind the latest Ecopro Co., Ltd. (086520.KQ) stock quote, history, news and other vital information to help you with your stock trading and investing."}, {'url': 'https://uk.finance.yahoo.com/quote/086520.KQ/', 'content': "You might not be getting everything you're entitled to — this is what you need t

'1. The latest trading day stock price for Ecopro (KOSDAQ: 086520.KQ) is 660,000 KRW. 2. South Korea totally banned short-selling on November 5, 2023. 3. Since the ban, the performance of Ecopro has decreased by approximately 20.29%.'

In [47]:
llm = ChatOpenAI(model="gpt-4", temperature=0, openai_api_key="")
tavily_tool = TavilySearchResults(api_wrapper=search, max_results=3)
tools = [Tool(
        name="Search",
        func=tavily_tool.run,
        description="useful for when you need to get ticker"
    )
]

ticker_agent = initialize_agent(tools=tools,
                          llm=llm,
                          agent=AgentType.OPENAI_FUNCTIONS, verbose=True)

In [48]:
ticker= ticker_agent.run("get me the ticker for Ecopro(KOSDAQ), you should return only ticker, no strings attached")



> Entering new AgentExecutor chain...

Invoking: `Search` with `Ecopro(KOSDAQ)`


[{'url': 'https://koreajoongangdaily.joins.com/2023/04/20/business/industry/Korea-ecopro-ecopro-bm/20230420174402783.html', 'content': "korea joongAng daily Home > Business > Industry [NEWS IN FOCUS] EcoPro stocks skyrocket, but some analysts spooked  EcoPro BM officials celebrate the company's public listing in March 2019 at Korea Exchange headquarters in Yeouido,  Climbing higher The price is right, EcoPro Materials CEO says ahead of IPO  EcoPro BM's building in Pohang, North Gyeongsang [YONHAP]EcoPro, a chemicals group that owns a battery materials unit, soared almost 500 percent in the first four months on the Kosdaq stock market, prompting investors and analysts to scrutinize the company. Solid demand and optimism for electric vehicles are major factors driving up the stock, but the breakneck rally has some analysts worried."}, {'url': 'https://finance.yahoo.com/quote/086520.KQ/', 'content': "Ecopr

In [49]:
ticker

'086520.KQ'